In [1]:
# -*- coding: utf-8 -*-
import sys
sys.path.append('/home/jun/SemanticReID')
import torch
from torch import nn, optim
from torch.nn import init
from torchvision import datasets, transforms 
from torchinfo import summary
from torch.nn import functional as F
import matplotlib
import time 
import os
import tqdm.auto as tqdm
from model import * 
from utils.random_erasing import RandomErasing
from utils.dgfolder import DGFolder
import yaml
from shutil import copyfile
from utils.circle_loss import CircleLoss, convert_label_to_similarity
from utils.instance_loss import InstanceLoss
from net.pspnet import PSPNet
from online_triplet_loss.losses import *
import timm
from utils.ODFA import ODFA
proxy = 'http://10.0.0.107:3128'
os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [ ]:
# Define the ViT Model with cls token
class ViTParsing(nn.Module):
    def __init__(self, class_num=751, droprate=0.5, stride=1, test=False, linear_num=512):
        super().__init__()
        PSP = PSPNet(sizes=(1, 2, 3, 6), psp_size=1024, deep_features_size=512, backend='densenet')
        PSP.load_state_dict(torch.load('../parsing_all.pth'))
        model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=class_num)
        self.patch_embed = model.patch_embed
        self.pos_drop = model.pos_drop
        self.cls_token = model.cls_token
        self.pos_embed = model.pos_embed
        self.patch_drop = model.patch_drop
        self.norm_pre = model.norm_pre
        self.blocks = model.blocks
        self.norm = model.norm
        self.fc_norm = model.fc_norm
        self.avgpool = nn.AdaptiveAvgPool2d((1,768))
        self.dropout = nn.Dropout(p=0.5)
        self.classifier = ClassBlock(768, class_num, droprate, linear=linear_num, test= test)
    def forward(self, x):
        x = self.patch_embed(x)
        cls_token = self.cls_token.expand(x.shape[0], -1, -1) 
        x = torch.cat((cls_token, x), dim=1)
        x = self.pos_drop(x + self.pos_embed)
        x = self.patch_drop(x)
        x = self.norm_pre(x)
        x = self.blocks(x) # torch.Size([32, 197, 768])
        x = self.norm(x) # torch.Size([32, 197, 768])
        x = self.fc_norm(x) # torch.Size([32, 197, 768]) 
        x = self.avgpool(x) # torch.Size([32, 197, 768]) 
        x = self.dropout(x)
        x = x.view(x.shape[0], x.shape[2])
        x = self.classifier(x)
        return x

In [3]:
model_train = ViT(702, linear_num=512)
#model_train.load_state_dict(torch.load('../model/LASwin_all_trick_duke/net_last.pth'))
#model_train.classifier.classifier = nn.Sequential() 
img = torch.rand((16,3,224,224)).to('cpu')
outputs = model_train(img)

In [24]:
#model_train = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)

Using cache found in /home/jun/.cache/torch/hub/pytorch_vision_v0.10.0


In [4]:
model_train

ViT(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2

In [6]:
summary(model=model_train, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

TypeError: 'bool' object is not callable